In [3]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *


training = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "sentimentdataset")

validation = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "validation")
testing = pd.read_excel(r"../../data/Des_SA/sentimentdataset.xlsx", "testing")

#remove nonASCIICharacters
training["Text"] = training.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)
validation["Text"] = validation.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)
testing["Text"] = testing.apply(lambda x: "".join([i for i in x["Text"] if ord(i) <= 127]), axis=1)






In [4]:
def to_class(s):
    match s:
        case "Positive":
            return 2
        case "Negative":
            return 1
        case "Neutral":
            return 0
    

In [5]:
training["SimpleSent"] = training.apply(lambda x: to_class(x["SimpleSent"]), axis=1)
testing["SimpleSent"] = testing.apply(lambda x: to_class(x["SimpleSent"]), axis=1)
validation["SimpleSent"] = validation.apply(lambda x: to_class(x["SimpleSent"]), axis=1)

In [6]:
toDrop = ["Timestamp", "User", "Platform", "Retweets", "Likes", "Country", "Year", "Month", "Day", "Hour"]

training = training.drop(toDrop, axis=1)
testing = testing.drop(toDrop, axis=1)
validation = validation.drop(toDrop, axis=1)


In [7]:
import re
def split_words(xs):
    empties = []
    empty = "" 
    for i in xs:
        words = re.split(r"([A-Z])", i)
        words = words[1::]
        for i in range(0, len(words) -1, 2):
            empty += words[i] + words[i + 1]
            empty += " "
        empties.append(empty.rstrip())
    return empties

In [8]:
from textblob import *
#process Hashtages
training["proc_hash"] = training.apply(lambda x: x["Hashtags"][1::], axis=1)
training["proc_hash"] = training.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
training["proc_hash"] = training.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
training["Hashtags"] = training["proc_hash"]
training = training.drop("proc_hash", axis=1)

testing["proc_hash"] = testing.apply(lambda x: x["Hashtags"][1::], axis=1)
testing["proc_hash"] = testing.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
testing["proc_hash"] = testing.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
testing["Hashtags"] = testing["proc_hash"]
testing = testing.drop("proc_hash", axis=1)

validation["proc_hash"] = validation.apply(lambda x: x["Hashtags"][1::], axis=1)
validation["proc_hash"] = validation.apply(lambda x: x["proc_hash"].replace(" #", " and "), axis=1)
validation["proc_hash"] = validation.apply(lambda x: " and ".join(split_words(x["proc_hash"].split(" and "))), axis=1)
validation["Hashtags"] = validation["proc_hash"]
validation = validation.drop("proc_hash", axis=1)

validation






,Text,Sentiment,SimpleSent,Hashtags
0,"After a series of defeats, the soccer team fac...",Disappointment,1,Disappointment and Disappointment Soccer Defeats
1,"In the tennis tournament, a highly anticipated...",Frustration,1,Frustration and Frustration Tennis Setback
2,"Facing a defeat in the championship, the boxer...",Reflection,0,Reflection and Reflection Boxing Defeat
3,"In the midst of a cycling race, a tire blowout...",Obstacle,0,Obstacle and Obstacle Cycling Frustration
4,The gymnast's unexpected fall during a routine...,Sympathy,2,Sympathy and Sympathy Gymnastics Fall
...,...,...,...,...
86,"Organized a community painting event, turning ...",Joy,2,Community Art and Community Art Senior Creativity
87,Hosted a 'memory lane' evening with old friend...,Gratitude,2,Friendship Adventures and Friendship Adventure...
88,"Joined a seniors' astronomy club, stargazing a...",Curiosity,0,Celestial Wonders and Celestial Wonders Senior...
89,"Attended a local jazz festival, tapping toes t...",Joy,2,Timeless Tunes and Timeless Tunes Senior Jazz


In [9]:


training["text_tb_pol"] = training.apply(lambda x: TextBlob(x["Text"]).polarity, axis=1)
training["text_tb_sub"] = training.apply(lambda x: TextBlob(x["Text"]).subjectivity, axis=1)

training["hash_tb_pol"] = training.apply(lambda x: TextBlob(x["Hashtags"]).polarity, axis=1)
training["hash_tb_sub"] = training.apply(lambda x: TextBlob(x["Hashtags"]).subjectivity, axis=1)




In [10]:
analyzer = SentimentIntensityAnalyzer()

training["text_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["Text"]), axis=1)
training["text_vader_neg"] = training.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
training["text_vader_neu"] = training.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
training["text_vader_pos"] = training.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
training = training.drop(["text_vader_scores"], axis=1)

training["hash_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["Hashtags"]), axis=1)
training["hash_vader_neg"] = training.apply(lambda x: x["hash_vader_scores"]["neg"], axis=1)
training["hash_vader_neu"] = training.apply(lambda x: x["hash_vader_scores"]["neu"], axis=1)
training["hash_vader_pos"] = training.apply(lambda x: x["hash_vader_scores"]["pos"], axis=1)
training = training.drop(["hash_vader_scores"], axis=1)







In [11]:
training

,Text,Sentiment,SimpleSent,Hashtags,text_tb_pol,text_tb_sub,hash_tb_pol,hash_tb_sub,text_vader_neg,text_vader_neu,text_vader_pos,hash_vader_neg,hash_vader_neu,hash_vader_pos
0,Enjoying a beautiful day at the park! ...,Positive,2,Nature and Nature Park,0.750000,0.800000,0.00,0.000000,0.000,0.397,0.603,0.0,1.000,0.000
1,Traffic was terrible this morning. ...,Negative,1,Traffic and Traffic Morning,-1.000000,1.000000,0.00,0.000000,0.437,0.563,0.000,0.0,1.000,0.000
2,Just finished an amazing workout! ...,Positive,2,Fitness and Fitness Workout,0.750000,0.900000,0.00,0.000000,0.000,0.494,0.506,0.0,0.323,0.677
3,Excited about the upcoming weekend getaway! ...,Positive,2,Travel and Travel Adventure,0.468750,0.750000,0.00,0.000000,0.000,0.650,0.350,0.0,0.566,0.434
4,Trying out a new recipe for dinner tonight. ...,Neutral,0,Cooking and Cooking Food,0.136364,0.454545,0.00,0.000000,0.000,1.000,0.000,0.0,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,Happy,2,Science Fair Winner and Science Fair Winner Hi...,0.875000,0.900000,0.52,0.780000,0.000,0.599,0.401,0.0,0.330,0.670
728,Attending a surprise birthday party organized ...,Happy,2,Surprise Celebration and Surprise Celebration ...,0.687500,0.600000,0.16,0.540000,0.000,0.327,0.673,0.0,0.413,0.587
729,Successfully fundraising for a school charity ...,Happy,2,Community Giving and Community Giving High Sch...,0.516667,0.383333,0.16,0.540000,0.000,0.468,0.532,0.0,0.556,0.444
730,"Participating in a multicultural festival, cel...",Happy,2,Cultural Celebration and Cultural Celebration ...,1.000000,1.000000,0.12,0.246667,0.000,0.479,0.521,0.0,1.000,0.000


In [12]:
'''
    Begin logistic regression
'''
'''
solver          penalty
lbfgs           l2
newton-cg       l1, l2
sag             l2
saga            elasticnet, l1, l2


'''

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

classes = training["SimpleSent"].to_numpy()
#features = training[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()

'''text only'''
features = training[["text_tb_pol", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()

#validate_features = testing[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()


#for i in range(100):

averages = []

total = 0
length = 0

#lbgfs
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
    lbgfs.fit(x_train, y_train)
    results = lbgfs.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

''''''
total = 0
length = 0
#newton-cg
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    newton_cg = LogisticRegression(penalty="l2", solver="newton-cg")
    newton_cg.fit(x_train, y_train)
    results = newton_cg.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

total = 0
length = 0
#sag
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    sag = LogisticRegression(penalty="l2", solver="sag", random_state=i)
    sag.fit(x_train, y_train)
    results = sag.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

total = 0
length = 0
#saga
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    saga = LogisticRegression(penalty="l2", solver="saga", random_state=i)
    saga.fit(x_train, y_train)
    results = saga.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)


averages





[0.8333636363636363,
 0.8334545454545453,
 0.8334545454545453,
 0.8334545454545453]

In [13]:
from sklearn.metrics import confusion_matrix
matrices = []
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
    lbgfs.fit(x_train, y_train)
    results = lbgfs.predict(x_test)
    actual = y_test
    matrices.append(confusion_matrix(y_test, results))

mean_matrix = np.mean(np.array(matrices), axis=0)
mean_matrix

array([[ 0.  ,  1.83, 10.26],
       [ 0.  , 24.87,  4.72],
       [ 0.  ,  1.52, 66.8 ]])

In [14]:
import pickle

x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=0)
lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
lbgfs.fit(x_train, y_train)
with open("TWEETS_LOG.pkl", "wb") as file:
    pickle.dump(lbgfs, file)
#results = lbgfs.predict(x_test)
#actual = y_test
#matrices.append(confusion_matrix(y_test, results))
